In [1]:
!pip install yfinance exa_py anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 8.5 MB/s eta 0:00:00


In [4]:
import re
import json
import requests
import yfinance as yf
import sqlite3
import datetime
from exa_py import Exa
import anthropic
from google.colab import userdata

# ----------------------
# Configuration / API Keys
# ----------------------
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
PERPLEXITY_API_KEY = userdata.get('PERPLEXITY_API_KEY')
EXA_API_KEY = userdata.get('EXA_API_KEY')

MODEL_NAME = "claude-3-5-sonnet-20241022"
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
exa = Exa(api_key=EXA_API_KEY)

# ----------------------
# Database Setup
# ----------------------
DB_FILE = "portfolio.db"

def init_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    # Create portfolio table
    c.execute('''CREATE TABLE IF NOT EXISTS portfolio (
                    ticker TEXT PRIMARY KEY,
                    shares INTEGER
                 )''')
    # Create trade orders log table
    c.execute('''CREATE TABLE IF NOT EXISTS trade_orders (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    ticker TEXT,
                    order_type TEXT,
                    shares INTEGER,
                    price REAL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
                 )''')
    # Initialize portfolio with starting positions if not already present
    starting_portfolio = {
        'AAPL': 50,
        'NVDA': 30,
        'TSLA': 20,
        'MSFT': 40
    }
    for ticker, shares in starting_portfolio.items():
        c.execute("INSERT OR IGNORE INTO portfolio (ticker, shares) VALUES (?, ?)", (ticker, shares))
    conn.commit()
    conn.close()

def update_portfolio_and_log_order(ticker, order_type, shares, price):
    """Simulate placing an order: update portfolio and log the order."""
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    # Fetch current shares
    c.execute("SELECT shares FROM portfolio WHERE ticker = ?", (ticker,))
    result = c.fetchone()
    current_shares = result[0] if result else 0

    if order_type.lower() == "buy":
        new_shares = current_shares + shares
    elif order_type.lower() == "sell":
        # Ensure we don't sell more than available
        if shares > current_shares:
            conn.close()
            return f"Error: Cannot sell {shares} shares of {ticker}; only {current_shares} available."
        new_shares = current_shares - shares
    else:
        # For "hold", do nothing.
        new_shares = current_shares

    c.execute("REPLACE INTO portfolio (ticker, shares) VALUES (?, ?)", (ticker, new_shares))
    c.execute("INSERT INTO trade_orders (ticker, order_type, shares, price) VALUES (?, ?, ?, ?)",
              (ticker, order_type, shares, price))
    conn.commit()
    conn.close()
    return f"Order processed: {order_type.upper()} {shares} shares of {ticker} at ${price:.2f}. New holding: {new_shares} shares."

# ----------------------
# Existing Data Fetching Functions
# ----------------------
def fetch_yahoo_finance(ticker):
    """Fetches last week's market data for a given ticker using yfinance."""
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="7d")
        if hist.empty:
            return f"Yahoo Finance: No market data found for {ticker} in the last week."
        return f"Yahoo Finance: Last week's market data for {ticker}:\n{hist.to_string()}"
    except Exception as e:
        return f"Error fetching Yahoo Finance data: {e}"

def fetch_perplexity_news(ticker):
    """Calls the Perplexity Chat Completions API to fetch recent news headlines for a ticker."""
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
         "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
         "Content-Type": "application/json"
    }
    payload = {
         "model": "sonar",
         "messages": [
              {"role": "system", "content": "Be precise and concise."},
              {"role": "user", "content": f"Provide the latest news headlines and summaries for {ticker}."}
         ],
         "max_tokens": 500,
         "temperature": 0.2,
         "top_p": 0.9,
         "top_k": 0,
         "stream": False,
         "presence_penalty": 0,
         "frequency_penalty": 1,
         "search_recency_filter": "week"
    }
    try:
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            data = response.json()
            answer = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return f"Perplexity News: {answer}"
        else:
            return f"Error fetching Perplexity news: {response.text}"
    except Exception as e:
        return f"Exception in Perplexity API call: {e}"

def fetch_exa_search(ticker, start_date, end_date, num_results=5):
    try:
        query = f"financial news and reports on {ticker}"
        result = exa.search_and_contents(
            query,
            text=True,
            num_results=num_results,
            start_published_date=start_date,
            end_published_date=end_date,
            include_text=[ticker]
        )
        results = result.results
        if results:
            entries = []
            for res in results:
                title = getattr(res, "title", None) or getattr(res, "url", "No title")
                content = getattr(res, "text", "No content available.")
                published_date = getattr(res, "published_date", "N/A")
                entry = (
                    f"Title: {title}\n"
                    f"Published: {published_date}\n"
                    f"Content: {content}"
                )
                entries.append(entry)
            return f"Exa Search: Found {len(entries)} results:\n" + "\n\n".join(entries)
        else:
            return f"Exa Search: No results found for {ticker} in the given period."
    except Exception as e:
        return f"Error fetching Exa Search results: {e}"

def calculate(expression):
    """Evaluates a sanitized arithmetic expression."""
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    try:
         result = eval(expression)
         return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
         return "Error: Invalid expression"

# ----------------------
# Tool Definitions (including new order_tool)
# ----------------------
tools = [
    {
        "name": "yahoo_finance",
        "description": (
            "Fetches current stock data for a given ticker from Yahoo Finance. "
            "Provides real-time pricing, volume, and market trends. "
            "Use this tool when you need quantitative data about a stock."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., TSLA for Tesla, Inc.)."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "perplexity_news",
        "description": (
            "Fetches recent news and financial reports for a given stock ticker using the Perplexity API. "
            "Use this tool to retrieve the latest headlines and summaries related to the stock."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., TSLA)."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "exa_search",
        "description": (
            "Performs a detailed search for financial news and reports on a given stock ticker using the Exa AI Search API. "
            "Use this tool when you need to fetch comprehensive search results, including reports within a specific date range."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., TSLA)."
                },
                "start_date": {
                    "type": "string",
                    "description": "The start date for the search in ISO format (e.g., '2024-11-15T08:00:00.000Z')."
                },
                "end_date": {
                    "type": "string",
                    "description": "The end date for the search in ISO format (e.g., '2025-02-16T07:59:59.999Z')."
                },
                "num_results": {
                    "type": "integer",
                    "description": "The number of search results to return.",
                    "default": 5
                }
            },
            "required": ["ticker", "start_date", "end_date"]
        }
    },
    {
        "name": "calculator",
        "description": (
            "A simple calculator that performs basic arithmetic operations. "
            "Use this tool to perform any necessary calculations (e.g., computing ratios or adjusting numbers)."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            "required": ["expression"]
        }
    },
    {
        "name": "order_tool",
        "description": (
            "Simulates placing a trade order by updating the portfolio and logging the trade. "
            "Order types can be 'buy' or 'sell' (for 'hold', no order is placed)."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                },
                "order_type": {
                    "type": "string",
                    "description": "The type of order: buy or sell."
                },
                "shares": {
                    "type": "integer",
                    "description": "The number of shares to trade."
                },
                "price": {
                    "type": "number",
                    "description": "The simulated price for the trade."
                }
            },
            "required": ["ticker", "order_type", "shares", "price"]
        }
    }
]

# ----------------------
# Processing Tool Calls
# ----------------------
def process_tool_call(tool_name, tool_input):
    if tool_name == "yahoo_finance":
        return fetch_yahoo_finance(tool_input["ticker"])
    elif tool_name == "perplexity_news":
        return fetch_perplexity_news(tool_input["ticker"])
    elif tool_name == "exa_search":
        ticker = tool_input["ticker"]
        start_date = tool_input["start_date"]
        end_date = tool_input["end_date"]
        num_results = tool_input.get("num_results", 5)
        return fetch_exa_search(ticker, start_date, end_date, num_results)
    elif tool_name == "calculator":
        return calculate(tool_input["expression"])
    elif tool_name == "order_tool":
        # Process the simulated trade order
        ticker = tool_input["ticker"]
        order_type = tool_input["order_type"]
        shares = tool_input["shares"]
        price = tool_input["price"]
        return update_portfolio_and_log_order(ticker, order_type, shares, price)
    else:
        return f"Error: Unknown tool {tool_name}"

# ----------------------
# Portfolio Reallocation Evaluation (Aggregate View)
# ----------------------
def evaluate_portfolio_reallocation():
    # Initialize the database and portfolio if needed
    init_db()

    # Retrieve the entire portfolio from the database
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("SELECT ticker, shares FROM portfolio")
    portfolio_data = c.fetchall()
    conn.close()

    # Build a summary string of the current portfolio
    portfolio_summary = "\n".join([f"{ticker}: {shares} shares" for ticker, shares in portfolio_data])

    # System prompt that instructs the AI to consider the overall portfolio
    system_prompt = (
        "You are an investment analysis assistant. Our current portfolio is:\n" +
        portfolio_summary +
        "\n\nBased on market data, news, and financial reports, please provide a comprehensive reallocation strategy. " +
        "Consider if adjustments are needed—such as selling some shares of one stock to buy more of another—to optimize our overall allocation. " +
        "You have access to tools to fetch market data (yahoo_finance), news (perplexity_news), detailed reports (exa_search), " +
        "and perform calculations (calculator). If you decide to place any trade orders, please use the order_tool. " +
        "The order_tool requires the following inputs: ticker, order_type (buy or sell), shares, and a simulated price. " +
        "Ensure that any sell orders do not exceed our current holdings."
    )

    # The initial conversation message for reallocation
    conversation = [
        {
            "role": "user",
            "content": (
                "Review our entire portfolio and propose a reallocation strategy. " +
                "Decide which stocks to buy or sell and by how many shares. " +
                "If you recommend any trades, please call the order_tool with the order details. " +
                "You may use the calculator tool for any arithmetic if needed."
            )
        }
    ]

    # Interaction loop: let the AI suggest a strategy and process any tool calls
    while True:
        response = client.messages.create(
            model=MODEL_NAME,
            system=system_prompt,
            max_tokens=4096,
            tools=tools,
            messages=conversation
        )

        conversation.append({
            "role": "assistant",
            "content": response.content
        })

        tool_called = False
        for block in response.content:
            if block.type == "tool_use":
                tool_called = True
                tool_name = block.name
                tool_input = block.input
                print(f"\n[Agent requests tool call] {tool_name} with input: {json.dumps(tool_input)}")
                result = process_tool_call(tool_name, tool_input)
                print(f"[Tool '{tool_name}' result] {result}\n")

                tool_result_message = {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": block.id,
                            "content": result
                        }
                    ]
                }
                conversation.append(tool_result_message)
                # Process one tool call at a time before getting a new response
                break

        # If no tool call was made, assume the AI has finished its recommendation
        if not tool_called:
            final_answer = ""
            for block in response.content:
                if block.type == "text":
                    final_answer += block.text
            print("\n=== Final Reallocation Strategy ===")
            print(final_answer)
            break

    # Print a summary of the updated portfolio and trade log
    print("\n=== Final Portfolio ===")
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("SELECT * FROM portfolio")
    for row in c.fetchall():
        print(row)
    print("\n=== Trade Orders Log ===")
    c.execute("SELECT * FROM trade_orders")
    for row in c.fetchall():
        print(row)
    conn.close()

# ----------------------
# Main
# ----------------------
if __name__ == "__main__":
    # Uncomment one of the following based on desired behavior:

    # 1. Evaluate stocks individually:
    # evaluate_portfolio_individually()

    # 2. Evaluate the entire portfolio for reallocation:
    evaluate_portfolio_reallocation()



[Agent requests tool call] yahoo_finance with input: {"ticker": "TSLA"}
[Tool 'yahoo_finance' result] Yahoo Finance: Last week's market data for TSLA:
                                 Open        High         Low       Close     Volume  Dividends  Stock Splits
Date                                                                                                         
2025-02-06 00:00:00-05:00  373.029999  375.399994  363.179993  374.320007   77918200        0.0           0.0
2025-02-07 00:00:00-05:00  370.190002  380.549988  360.339996  361.619995   70298300        0.0           0.0
2025-02-10 00:00:00-05:00  356.209991  362.700012  350.510010  350.730011   77514900        0.0           0.0
2025-02-11 00:00:00-05:00  345.799988  349.369995  325.100006  328.500000  118543400        0.0           0.0
2025-02-12 00:00:00-05:00  329.940002  346.399994  329.119995  336.510010  105382700        0.0           0.0
2025-02-13 00:00:00-05:00  345.000000  358.690002  342.850006  355.940002   89